<a href="https://colab.research.google.com/github/AI-Samoyed/COVID-Machine-Learning-/blob/master/COVID_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Packages to install

!pip install geojson
!pip install geopandas

import requests
import folium
import geojson
import geopandas as gpd
import urllib.request
import json
import branca.colormap as cm
from folium.plugins import TimeSliderChoropleth

# General Tools
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# regression functions
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# DATA:
# from "https://resources-covid19canada.hub.arcgis.com/datasets/PHAC-ASPC::covid19-data"
with urllib.request.urlopen("https://opendata.arcgis.com/datasets/a38f44f1a9c14fe1b8352933c71731de_0.geojson") as url:
    data = json.loads(url.read().decode())

# polygon files for the map
map_url = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/canada.geojson"
with urllib.request.urlopen(map_url) as url:
    map_data = gpd.read_file(url)

feature_data = pd.read_csv('https://raw.githubusercontent.com/AI-Samoyed/COVID-Machine-Learning-/master/feature2')

     |████████████████████████████████| 972kB 7.6MB/s 
     |████████████████████████████████| 14.8MB 250kB/s 
     |████████████████████████████████| 6.5MB 43.8MB/s 


In [2]:
raw = pd.DataFrame.from_dict(data["features"])
raw = pd.DataFrame.from_records(raw["properties"])
raw

,PRUID,Name,NameFR,Date,Confirmed_Cases,Probable_Cases,Deaths,Total_Cases,Cases_Today,Percent_Today,ObjectId,Deaths_Today,Current_Day,Number_Recovered,Number_Tested,RecoveredToday,TestedToday,Precent_Recovered,Percent_Death,Percent_Active
0,35,Ontario,Ontario,2020/01/31 12:00:00+00,3,0,0,3,3,300.00,1,0,0,0,0,0,0,NaN,0.00,100.00
1,59,British Columbia,Colombie-Britannique,2020/01/31 12:00:00+00,1,0,0,1,1,100.00,2,0,0,0,0,0,0,NaN,0.00,100.00
2,1,Canada,Canada,2020/01/31 12:00:00+00,4,0,0,4,4,400.00,3,0,0,0,0,0,0,NaN,0.00,100.00
3,35,Ontario,Ontario,2020/02/08 12:00:00+00,3,0,0,3,0,0.00,4,0,0,0,0,0,0,NaN,0.00,100.00
4,59,British Columbia,Colombie-Britannique,2020/02/08 12:00:00+00,4,0,0,4,3,300.00,5,0,0,0,0,0,0,NaN,0.00,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,48,Alberta,Alberta,2020/11/25 12:00:00+00,50801,0,500,50801,1265,2.55,3953,8,1,36582,1426119,887,5871,72.01,0.98,27.01
3953,59,British Columbia,Colombie-Britannique,2020/11/25 12:00:00+00,29086,0,371,29086,738,2.60,3954,13,1,19814,761442,209,6871,68.12,1.28,30.60
3954,99,Repatriated travellers,Voyageurs rapatriés,2020/11/25 12:00:00+00,13,0,0,13,0,0.00,3955,0,1,13,76,0,0,100.00,0.00,0.00
3955,10,Newfoundland and Labrador,Terre-Neuve-et-Labrador,2020/11/25 12:00:00+00,324,0,4,324,1,0.31,3956,0,1,295,60199,0,458,91.05,1.23,7.72


In [3]:
ON_features = feature_data[feature_data['Province_Name'] == 'Ontario']
QC_features = feature_data[feature_data['Province_Name'] == 'Quebec']
AB_features = feature_data[feature_data['Province_Name'] == 'Alberta']
BC_features = feature_data[feature_data['Province_Name'] == 'British Columbia']
NS_features = feature_data[feature_data['Province_Name'] == 'Nova Scotia']
YU_features = feature_data[feature_data['Province_Name'] == 'Yukon']
NW_features = feature_data[feature_data['Province_Name'] == 'Northwest Territories']
NV_features = feature_data[feature_data['Province_Name'] == 'Nunavut']
PEI_features = feature_data[feature_data['Province_Name'] == 'Prince Edward Island']
MB_features = feature_data[feature_data['Province_Name'] == 'Manitoba']
SS_features = feature_data[feature_data['Province_Name'] == 'Saskatchewan']
NL_features = feature_data[feature_data['Province_Name'] == 'Newfoundland and Labrador']
NB_features = feature_data[feature_data['Province_Name'] == 'New Brunswick']

In [4]:
def create_rolling_window(data, window_size=7):
    """ 
        Create a window from time A to B where each entry in the window is the
        number of new cases on that day.
    """
    #data = data[data > 0] # remove days missing data
    ws = window_size + 1
    x_data = data[:-1]
    
    x_data = np.reshape(x_data, (-1,1))
    temp_data = np.roll(x_data, -1)
    for x in range(1, ws):
        # Stack
        x_data = np.concatenate((x_data[:-1], temp_data[:-1]), 1)
        # Step, roll left by 1
        temp_data = np.roll(temp_data, -1)[:-1]
    
    cols = ["day{}".format(i) for i in range(1,ws)] + ['newDay']
    return pd.DataFrame(x_data, columns=cols)

def split_train_test(data):
    """
        Divide into train,test,val sets
        Take first 70% as train and next 30% as test
    """
    ntrain,ntest = int(len(data) * 0.7), int(len(data) * 0.1)
    data = data.sample(frac=1)
    
    train = data[:ntrain].reset_index(drop=True)
    test = data[ntrain:].reset_index(drop=True)
    return train, test

def add_features(df,features):
    """ Append feature data to every row of df. """
    df['merger'] = 1
    features['merger'] = 1
    
    df = df.merge(features, how='outer', on='merger').drop(['merger','Province_Name'], axis=1)
    return df

def prepare_data(covid_df, features):
    """ Process of creating the final training and test sets from initial data. """
    rw = create_rolling_window(covid_df.values)
    rw_with_features = add_features(rw,features)
    
    return split_train_test(rw_with_features)

def split_xy(data):
    """ Separate training features and training output. """
    return data[data.columns.difference(['newDay'])], data['newDay']

In [5]:
def model_with_RF(train_data,test_data,start_date):
    """ Model data using RandomForest Regressor. """
    x_train, y_train = split_xy(train_data)
    x_test, y_test = split_xy(test_data)
    
    model = RandomForestRegressor(n_estimators = 100, random_state = 0)
    model.fit(x_train,y_train)
    
    y_preds = model.predict(x_test)
    date_range = pd.date_range(start=start_date, end=pd.to_datetime("today"), periods=(len(y_preds)))

    df = pd.DataFrame({"Date": date_range, "Actual":y_test,"Preds":y_preds})
    df['Date'] = df['Date'].dt.date

    #print(np.mean(np.abs(y_preds-y_test))) # mean absolute error
    #print(np.mean(np.power(y_preds-y_test,2))) # mean squared error

    #plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    #plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    #plt.plot(df['Date'], df['Preds'], label='preds', lw=2, marker='o')
    #plt.plot(df['Date'], df['Actual'], label='actual', lw=2, marker='s')  
    #plt.gcf().autofmt_xdate()
    #plt.legend()
    return df
    


def model_with_LR(train_data,test_data,start_date):
    """ Model data using LinearRegressor. """
    x_train, y_train = split_xy(train_data)
    x_test, y_test = split_xy(test_data)
    
    model = LinearRegression()
    model.fit(x_train,y_train)
    
    y_preds = model.predict(x_test)
    date_range = pd.date_range(start=start_date, end=pd.to_datetime("today"), periods=(len(y_preds)))
    
    df = pd.DataFrame({"Date": date_range, "Actual":y_test,"Preds":y_preds})
    df['Date'] = df['Date'].dt.date

    #print(np.mean(np.abs(y_preds-y_test))) # mean absolute error
    #print(np.mean(np.power(y_preds-y_test,2))) # mean squared error

    #plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    #plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    #plt.plot(df['Date'], df['Preds'], label='preds', lw=2, marker='o')
    #plt.plot(df['Date'], df['Actual'], label='actual', lw=2, marker='s')  
    #plt.gcf().autofmt_xdate()
    #plt.legend()
    return df
    print(np.mean(np.abs(y_preds-y_test))) # mean absolute error
    print(np.mean(np.power(y_preds-y_test,2))) # mean squared error


    
    

In [6]:
def roll_days(df):
    """ Shift all case data one day to the left. """
    day = 'day{}'
    for i in range(1,7):
        df[day.format(i)] = df[day.format(i+1)]
        
    return df
        
def forecast(case_data, feature_data, days=7):
    """ Recursively predict the next 'days' by including the last prediction. """
    forecast_train_data = case_data[:-days]
    forecast_test_data = case_data[-days:].reset_index(drop=True)
    
    rw = create_rolling_window(forecast_train_data.values)
    rw_plus_features = add_features(rw,feature_data)
    forecast_start = rw_plus_features[-1:]
    X,Y = split_xy(rw_plus_features)
    
    model = LinearRegression()
    model.fit(X,Y)
    
    forecast_start = roll_days(forecast_start)
    forecast_start['day7'] = forecast_start['newDay']
    forecast_start, _ = split_xy(forecast_start) # drop the 'newDay' column

    dcols = ['day1','day2','day3','day4','day5','day6','day7',]
    predictions = []
    for i in range(days):
        predictions.append(round(model.predict(forecast_start)[0]))
        forecast_start = roll_days(forecast_start)
        forecast_start['day7'] = predictions[-1]
        
    print(predictions)
    print(forecast_test_data)
    print(np.mean(np.abs(predictions-forecast_test_data))) # mean absolute error
    print(np.mean(np.power(predictions-forecast_test_data,2))) # mean squared error
    
    plt.plot(predictions, label='preds')
    plt.plot(forecast_test_data, label='actual')
    plt.legend()

In [10]:
def makemap(dataframe):
  cmap = cm.linear.YlOrRd_09.scale(0, max(dataframe['Preds']))
  dataframe['colour'] = dataframe['Preds'].map(cmap)
  prov_list = dataframe['name'].unique().tolist()
  prov_idx = range(len(prov_list))

  style_dict = {}
  for i in prov_idx:
      prov = prov_list[i]
      result = dataframe[dataframe['name'] == prov]
      inner_dict = {}
      for _, r in result.iterrows():
          inner_dict[r['Date']] = {'color': r['colour'], 'opacity': 0.7}
      style_dict[str(i)] = inner_dict
    
  prov_df = dataframe[['geometry']]
  prov_gdf = gpd.GeoDataFrame(prov_df)
  prov_gdf = prov_gdf.drop_duplicates().reset_index()
  return prov_gdf,  style_dict, cmap

In [27]:
raw_test = raw.copy()
raw_exclude = (raw_test['Name'] != "Canada") & (raw_test['Name'] != "Repatriated travellers")
raw_test = raw_test.loc[raw_exclude]

prov_names = (raw_test["Name"].unique()).tolist()

# **Actual Case Today**

In [34]:
def makemap_org(dataframe):
  cmap = cm.linear.YlOrRd_09.scale(0, max(dataframe['Cases_Today']))
  dataframe['colour'] = dataframe['Cases_Today'].map(cmap)
  prov_list = dataframe['name'].unique().tolist()
  prov_idx = range(len(prov_list))

  style_dict = {}
  for i in prov_idx:
      prov = prov_list[i]
      result = dataframe[dataframe['name'] == prov]
      inner_dict = {}
      for _, r in result.iterrows():
          inner_dict[r['Date']] = {'color': r['colour'], 'opacity': 0.7}
      style_dict[str(i)] = inner_dict
    
  prov_df = dataframe[['geometry']]
  prov_gdf = gpd.GeoDataFrame(prov_df)
  prov_gdf = prov_gdf.drop_duplicates().reset_index()
  return prov_gdf,  style_dict, cmap

raw_test_df = raw_test.copy()
raw_test_df = raw_test_df.rename(columns={'Name': 'name'})

raw_test_df ['Date'] = pd.to_datetime(raw_test_df['Date']).astype(int) / 10**9
raw_test_df ['Date'] = raw_test_df['Date'].astype(int).astype(str)
raw_test_df = raw_test_df[["name", "Date", "Cases_Today"]]

raw_map_df = raw_test_df.merge(map_data, on='name')
raw_map_df = raw_map_df[['name', 'Date', 'Cases_Today', 'geometry']]
raw_map_df #the input dataframe must look like this

raw_prov_gdf, raw_style_dict, raw_cmap= makemap_org(raw_map_df)

### Map Part

raw_map = folium.Map(location=[56.1304, -106.3468], 
                     zoom_start=2.5, min_zoom=2, width=1000, height=500, max_bounds=True)

_ = TimeSliderChoropleth(
      data=raw_prov_gdf.to_json(),
      styledict=raw_style_dict,
  ).add_to(raw_map)

_ = raw_cmap.add_to(raw_map)

raw_cmap.caption = "RF Pred Today"

folium.LayerControl().add_to(raw_map)

raw_map

# **Random Forest**

In [28]:
def all_prov_RF(names, main_data, feature_data, start_date):
  main_result = pd.DataFrame({})

  for prov_name in names:
    print(f"working on {prov_name}")
    cur_covid = main_data[main_data['Name'] == prov_name]
    cur_covid = cur_covid[(cur_covid['Date'] > start_date )] 
    cols = ['Date', 'Name', 'Cases_Today']
    cur_covid = cur_covid[cols].reset_index()
    cur_covid_processed = cur_covid[cur_covid['Name'] == prov_name]['Cases_Today'].reset_index(drop=True)
    feature_set = feature_data[feature_data['Province_Name'] == prov_name]
    prov_train, prov_test = prepare_data(cur_covid_processed, feature_set)
    prov_result = model_with_RF(prov_train,prov_test,start_date)
    prov_result['name']=prov_name
    main_result = pd.concat([main_result, prov_result], axis=0)
  
  return main_result

all_prov_RF = all_prov_RF(prov_names, raw_test, feature_data, start_date)
all_prov_RF

working on Ontario
working on British Columbia


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


working on Quebec


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

working on Alberta
working on Northwest Territories
working on Nunavut


...

working on Nova Scotia
working on New Brunswick


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


working on Saskatchewan
working on Newfoundland and Labrador
working on Manitoba


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Date,Actual,Preds,name
0,2020-03-11,164,130.78,Ontario
1,2020-03-14,1248,1349.86,Ontario
2,2020-03-18,338,374.94,Ontario
3,2020-03-22,483,503.73,Ontario
4,2020-03-25,169,124.71,Ontario
...,...,...,...,...
71,2020-11-27,12,23.13,Manitoba
72,2020-11-30,1,0.99,Manitoba
73,2020-12-04,1,1.03,Manitoba
74,2020-12-08,65,34.06,Manitoba


In [29]:
RF_test_df = all_prov_RF.copy()

RF_test_df ['Date'] = pd.to_datetime(RF_test_df['Date']).astype(int) / 10**9
RF_test_df ['Date'] = RF_test_df['Date'].astype(int).astype(str)
RF_test_df = RF_test_df[["name", "Date", "Preds"]]

RF_map_df = RF_test_df.merge(map_data, on='name')
RF_map_df = RF_map_df[['name', 'Date', 'Preds', 'geometry']]
RF_map_df #the input dataframe must look like this

RF_prov_gdf, RF_style_dict, RF_cmap= makemap(RF_map_df)

### Map Part


RF_map = folium.Map(location=[56.1304, -106.3468], 
                     zoom_start=2.5, min_zoom=2, width=1000, height=500, max_bounds=True)

_ = TimeSliderChoropleth(
      data=RF_prov_gdf.to_json(),
      styledict=RF_style_dict,
  ).add_to(RF_map)

_ = RF_cmap.add_to(RF_map)

RF_cmap.caption = "RF Pred Today"

folium.LayerControl().add_to(RF_map)

RF_map

# **Linear Regression**

In [ ]:
def all_prov_LR(names, main_data, feature_data, start_date):
  main_result = pd.DataFrame({})

  for prov_name in names:
    print(f"working on {prov_name}")
    cur_covid = main_data[main_data['Name'] == prov_name]
    cur_covid = cur_covid[(cur_covid['Date'] > start_date )] 
    cols = ['Date', 'Name', 'Cases_Today']
    cur_covid = cur_covid[cols].reset_index()
    cur_covid_processed = cur_covid[cur_covid['Name'] == prov_name]['Cases_Today'].reset_index(drop=True)
    feature_set = feature_data[feature_data['Province_Name'] == prov_name]
    prov_train, prov_test = prepare_data(cur_covid_processed, feature_set)
    prov_result = model_with_LR(prov_train,prov_test,start_date)
    prov_result['name']=prov_name
    main_result = pd.concat([main_result, prov_result], axis=0)
  
  return main_result

all_prov_LR = all_prov_LR(prov_names, raw_test, feature_data, start_date)
all_prov_LR

In [39]:
LR_test_df = all_prov_LR.copy()

LR_test_df ['Date'] = pd.to_datetime(LR_test_df['Date']).astype(int) / 10**9
LR_test_df ['Date'] = LR_test_df['Date'].astype(int).astype(str)
LR_test_df = LR_test_df[["name", "Date", "Preds"]]

LR_map_df = LR_test_df.merge(map_data, on='name')
LR_map_df = LR_map_df[['name', 'Date', 'Preds', 'geometry']]
LR_map_df #the input dataframe must look like this

LR_prov_gdf, LR_style_dict, LR_cmap= makemap(RF_map_df)

### Map Part

LR_map = folium.Map(location=[56.1304, -106.3468], 
                     zoom_start=2.5, min_zoom=2, width=1000, height=500, max_bounds=True)

_ = TimeSliderChoropleth(
      data=LR_prov_gdf.to_json(),
      styledict=LR_style_dict,
  ).add_to(LR_map)

_ = LR_cmap.add_to(LR_map)

LR_cmap.caption = "LR Pred Today"

folium.LayerControl().add_to(LR_map)

LR_map

# **LTSM**

In [41]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

def lstm_simple(data, start_date_select='2020-3-11', no_zero = 1, n_days_in = 14, n_days_out = 1):

  '''
  Input:

  Output:
  result: dataframe including all provinces
  '''
  def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps_in
      out_end_ix = end_ix + n_steps_out
      # check if we are beyond the sequence
      if out_end_ix > len(sequence):
        break
      # gather input and output parts of the pattern
      seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
      X.append(seq_x)
      y.append(seq_y)
    return np.array(X), np.array(y)
  
  data = data[["Name", "Date", "Cases_Today", "RecoveredToday", "TestedToday"]]
  raw_main = data.copy()

  pd.DataFrame.from_records(raw_main)

  #get all province name
  prov_names = (raw_main["Name"].unique()).tolist()

  #collect all province in main_result
  main_result = pd.DataFrame({})

  for prov_name in prov_names:
    raw = raw_main[raw_main["Name"] == prov_name]
    print(f"working on {prov_name}")
    # select based on date and excluding zeros or not
    start_date = (data['Date'] >= start_date_select)
    raw = raw.loc[start_date]

    if (no_zero == 1):
      no_zero_row = (raw['Cases_Today'] != 0)
      raw = raw.loc[no_zero_row]
    
    raw_seq = raw['Cases_Today'].tolist()
    date_seq = raw['Date'].tolist()

    # choose a number of time steps
    n_steps_in, n_steps_out = n_days_in, n_days_out
    # split into samples
    X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = 1
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    # define model
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=50, verbose=0)
    # demonstrate prediction
    result = []
    date_list = []
    for i in range(len(raw_seq) - n_steps_in):
      x_input = np.array(raw_seq[i:i+n_steps_in])
      x_input = x_input.reshape((1, n_steps_in, n_features))
      yhat = model.predict(x_input, verbose=0)
      result.append(yhat.item())
      date_list.append(date_seq[i+n_steps_in])
      #print(x_input, date_seq[i+n_steps_in], yhat.item())
    result = [round(x) for x in result]

    df_test = pd.DataFrame({'Date': date_list, 'Preds': result}) 
    start_date_select_df = (pd.to_datetime(start_date_select) + pd.DateOffset(days=n_steps_in))
    df_true = raw.loc[raw['Date'] >= start_date_select_df.strftime('%Y-%m-%d')]
    df_combine = (df_true.merge(df_test, left_index=False, right_index=False, how='inner'))
    df_combine['Difference'] = df_combine['Preds'] - df_combine['Cases_Today']

    main_result = pd.concat([main_result, df_combine], axis=0)

  return main_result

In [42]:
test_lstm = lstm_simple(raw_test, start_date_select='2020-3-11', no_zero = 1, n_days_in = 7, n_days_out = 1)
test_lstm

working on Ontario
working on British Columbia
working on Quebec
working on Alberta
working on Northwest Territories
working on Nunavut
working on Prince Edward Island
working on Yukon
working on Nova Scotia
working on New Brunswick
working on Saskatchewan
working on Newfoundland and Labrador
working on Manitoba


,Name,Date,Cases_Today,RecoveredToday,TestedToday,Preds,Difference
0,Ontario,2020/03/05 12:00:00+00,2,0,0,5,3
1,Ontario,2020/03/06 12:00:00+00,6,0,0,5,-1
2,Ontario,2020/03/08 12:00:00+00,3,0,0,7,4
3,Ontario,2020/03/09 12:00:00+00,3,0,0,10,7
4,Ontario,2020/03/11 12:00:00+00,8,0,3394,14,6
...,...,...,...,...,...,...,...
196,Manitoba,2020/11/21 12:00:00+00,385,224,0,367,-18
197,Manitoba,2020/11/22 12:00:00+00,240,118,0,430,190
198,Manitoba,2020/11/23 12:00:00+00,543,160,9019,433,-110
199,Manitoba,2020/11/24 12:00:00+00,471,280,3646,461,-10


In [45]:
lstm_df = test_lstm.copy()
lstm_df = lstm_df.rename(columns={'Name': 'name'})
lstm_df 

lstm_df ['Date'] = pd.to_datetime(lstm_df['Date']).astype(int) / 10**9
lstm_df ['Date'] = lstm_df['Date'].astype(int).astype(str)
lstm_df = lstm_df[["name", "Date", "Preds"]]

lstm_map_df = lstm_df.merge(map_data, on='name')
lstm_map_df = lstm_map_df[['name', 'Date', 'Preds', 'geometry']]
lstm_map_df

lstm_prov_gdf, lstm_style_dict, lstm_cmap= makemap(lstm_map_df)

### Map Part

lstm_map_df = folium.Map(location=[56.1304, -106.3468], 
                     zoom_start=2.5, min_zoom=2, width=1000, height=500, max_bounds=True)

_ = TimeSliderChoropleth(
      data=lstm_prov_gdf.to_json(),
      styledict=lstm_style_dict,
  ).add_to(lstm_map_df)

_ = lstm_cmap.add_to(lstm_map_df)

lstm_cmap.caption = "LSTM Pred Today"

folium.LayerControl().add_to(lstm_map_df)

lstm_map_df